<a href="https://colab.research.google.com/github/Nuthan03/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intorduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language(could be seqeuences of texts or speech).
Another common term for NLP problems is sequence to sequence problems(seq2seq)

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## Get the helper functions


In [2]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper function for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-12-13 03:08:10--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-13 03:08:10 (73.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
The dataset we are going to be using us Kaggle's introduction to NLP datasets (text samples of Tweets labelled as diaster or not diaster).

In [3]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-13 03:08:15--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.120.207, 142.251.161.207, 74.125.126.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.120.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-12-13 03:08:15 (125 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/


In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real diaster)
Text:
This guy idk just made me his woman crush ?? first one ever ??

---

Target: 0 (not real diaster)
Text:
seeing as how this person is a mass murderer and has like FBI CIA grade equipment to do evil shit.

---

Target: 1 (real diaster)
Text:
USGS EQ: M 0.6 - 8km SSW of Anza California: Time2015-08-06 01:26:24 UTC2015-08-05 18:26:24 -07:0... http://t.co/3bwWNLsxhB #EarthQuake

---

Target: 0 (not real diaster)
Text:
The Sound of Arson

---

Target: 0 (not real diaster)
Text:
just completely annihilated cech with paul keegan what a time to be alive

---



## Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this namely:
* Tokenziation-direct mapping of token(a token could be a word or a character) to number
* Embedding-create a martrix of feature vector for each token(the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization(tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # How many words in the vocabullary
                                    standardize="lower_and_strip_punctuation", # How to process text
                                    split="whitespace", # How to split tokens
                                    ngrams=None, # Grouping words is none
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # How long should the output sequence of tokens be?
                                    #pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# Setup text vectorization wtth custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [18]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [19]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 San Antonio Stars head coach Dan Hughes was just carted to the locker room after one of his guards collided with... http://t.co/4dbhOnO3Rk        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1483, 6221, 1969,  331, 3994, 1308, 2938,   23,   29,    1,    5,
           2, 3667, 1181,   43]])>

In [21]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we are going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:


*   `input_dim` = the size of our vocabulary.
*   `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
*   `input_length` = length of the sequences being passed to the embedding layer



In [23]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform",
                             input_length=max_length, # how long is each input
                             name="embedding")

embedding


In [29]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
       \n\nEmbedded version")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
#CDCwhistleblower 2 deaths from measles complications in 10 yrs everyone looses their shit. 8 dead from Legionnaires in a month &amp; crickets       

Embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02289406,  0.04828024,  0.04191415, ...,  0.03901639,
         -0.00107503, -0.01809304],
        [ 0.03554057,  0.02203143,  0.04126306, ...,  0.03097341,
          0.0432849 , -0.01316303],
        [-0.02876612,  0.04958062, -0.03689367, ...,  0.01614456,
         -0.04464313, -0.00279076],
        ...,
        [ 0.02052213, -0.0202678 , -0.0255337 , ..., -0.00786411,
         -0.04562432,  0.04675872],
        [ 0.00682815, -0.01104105,  0.02279424, ...,  0.02067823,
         -0.01187507, -0.04050039],
        [ 0.04051209,  0.02599313,  0.03540224, ..., -0.00280134,
         -0.0336651 ,  0.0227613 ]]], dtype=float32)>

In [30]:
# Chcek out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02289406,  0.04828024,  0.04191415,  0.04097148, -0.00634484,
        -0.00389787,  0.02222968,  0.04048179,  0.04223745, -0.02642385,
        -0.01930555,  0.04801276,  0.04030049,  0.00300574, -0.04361013,
        -0.03239658,  0.03012062,  0.04941744,  0.00665015, -0.04766807,
         0.02452243, -0.02588422, -0.03012588,  0.04937688, -0.0450935 ,
         0.04556917,  0.03844522,  0.0241839 , -0.02207953, -0.01468569,
        -0.01668756, -0.02186071, -0.0202132 ,  0.0017659 ,  0.00456335,
         0.00427269,  0.00498587, -0.034674  ,  0.03016999, -0.04622296,
        -0.01913174, -0.04585641, -0.03294472,  0.0252409 ,  0.0471318 ,
        -0.0016894 , -0.02594011, -0.01641351, -0.01930996,  0.00672164,
        -0.04819894,  0.03369415, -0.01693366,  0.02657013, -0.01639564,
        -0.01381712,  0.0097761 , -0.04588788, -0.0270667 ,  0.02937951,
         0.00364292,  0.03392193,  0.04456467,  0.0411291 ,  0.00988722,
  

## Modelling a text dataset

Now we have got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Navie Bayes(baseline), this is from Sklearn ML map.
* Model 1: Feed-forward neural network(dense model)
* Model 2: LSTM model(RNN)
* Model 3: GRU model(RNN)
* Model 4: Bidirectional-LSTM model(RNN)
* Model 5: 1D Convolutional Neural Network(CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor(using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to apporach all these 6 model?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate our model